<a href="https://colab.research.google.com/github/anju1982/Anju/blob/main/Transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#transfer learning
# Retrieve the Xception model from Keras
#view the entire model  
Xception_model = keras.applications.Xception()

tf.keras.utils.plot_model(#before using this model we want to know what the Xception model is
    Xception_model,
    to_file='Xmodel.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)

In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image
     

2.9.2


In [26]:
 ! unzip '/content/drive/MyDrive/MLand DL/Flower_dataset.zip' -d '/content/drive/MyDrive/ML and DL'

unzip:  cannot find or open /content/drive/MyDrive/MLand DL/Flower_dataset.zip, /content/drive/MyDrive/MLand DL/Flower_dataset.zip.zip or /content/drive/MyDrive/MLand DL/Flower_dataset.zip.ZIP.


In [25]:
imgFiles = glob.glob("/content/drive/MyDrive/mlanddl/Flower_dataset/*/*.jpg") #navigate through sub-folders, parses them and store in a list
for items in imgFiles[:8]: #imgFiles is a list of strings, each string corresponds to the file path 
  print(items)

In [22]:
print(len(imgFiles))

0


In [10]:
X = []
y = []

for fName in imgFiles:
  
  # Prepare the dataset and populate X and y
  X_i = Image.open(fName) # tiny_FR/sunflower/1715303025_e7065327e2.jpg (500, 333)
  X_i = X_i.resize((299,299)) # To make them approriate to Xception model when using Transfer Learning 
  X_i = np.array(X_i) / 255.0 # Normalize to range 0.0 to 1.0 
  X.append(X_i)

  label = fName.split("/") # ['tiny_FR', 'sunflower', '1715303025_e7065327e2.jpg'], list containing substrings
  y_i = label[-2] # 'sunflower'
 
  y.append(y_i)

#processed all  image files and corresponding intensity values and labels stored in X and y respectively 

In [11]:
print(set(y)) # python function that lists unique values in the list. # these are string labels

# but the network can only deal with numeric data

set()


In [12]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)
print(set(y))

print(lEncoder.classes_) #classes_ is a member variable in the class

set()
[]


In [13]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)


(0,)
(0,)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    stratify=y, random_state=42)

print("X_train_shape: {}".format(X_train.shape))
print("X_test_shape: {}".format(X_test.shape))
     

ValueError: ignored

In [ ]:
#Keras Applications
#https://keras.io/api/applications/
#Keras Applications
# Retrieve the Xception model from Keras
#view the entire model  
Xception_model = keras.applications.Xception()

tf.keras.utils.plot_model(
    Xception_model,
    to_file='Xmodel.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)

In [ ]:
#For transfer learning
include_top=False
weights = 'imagenet
##https://keras.io/api/applications/


In [ ]:
# Take only the feature extractor part

base_model = keras.applications.Xception(weights='imagenet', 
                                                  include_top=False)

# Visualize the extractor part for transfer learning

tf.keras.utils.plot_model(
    base_model,
    to_file='baseModel.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)

In [ ]:
# Make the weights/parameters of fetaure extractor non-trainable
# Freeze the weights - will not be updated during backpropagation
for layer in base_model.layers:
  layer.trainabe = False

In [ ]:
#Add classifier part
Global Average Pooling - 2D
Output layer (5 units (because 5-class classificaiton problem)
# Adding classifier
#Inspect how flattening is made in the pretrained model. In Xception, they use GAP
global_pool = keras.layers.GlobalAveragePooling2D()(base_model.output)

output_ = keras.layers.Dense(units=5, activation='softmax')(global_pool)

model_TL = keras.models.Model(inputs=[base_model.input], outputs=[output_])

# Visualize the complete model
tf.keras.utils.plot_model(
    model_TL,
    to_file='Model_TL.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)
     

In [ ]:
Compile and train the model
Save the best weights while training
https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
model_TL.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])
#introduce callbacks to save the best model


callbacks_TL = [            
             keras.callbacks.ModelCheckpoint("bestTL.h5",
                                             monitor='val_accuracy',verbose=1,
                                             save_weights_only=True,
                                             save_best_only=True)]


history_TL = model_TL.fit(x = X_train_std, y = y_train, epochs=50, 
                          validation_split=0.1, batch_size=16, callbacks=callbacks_TL)
     

keys = ['accuracy', 'val_accuracy']
progress = {k:v for k,v in history_TL.history.items() if k in keys}

import pandas as pd
pd.DataFrame(progress).plot()

plt.xlabel("epochs")
plt.ylabel("accuracy")

plt.grid(True)
plt.show()
     


In [ ]:
#Evaluate the model 

testLoss_TL, testAccuracy_TL = model_TL.evaluate(x = X_test_std, y = y_test)
print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

In [ ]:
Update the model with best weights
These weights may be from an intermediate epoch

In [ ]:
model_TL.load_weights("bestTL.h5")

testLoss_TL, testAccuracy_TL = model_TL.evaluate(x = X_test_std, y = y_test)

print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

In [ ]:
#Save the model & dataset(for finetuning)
# Saves the best model obtained in drive( if not saved in drive)
model_TL.save('/content/drive/MyDrive/Models/01_Xception_TransferLearning_Best_Model.h5')
     

In [ ]:
from numpy import save
#.npy is numpy array format 
save('/content/drive/MyDrive/Models/X_train_std.npy', X_train_std)
save('/content/drive/MyDrive/Models/X_test_std.npy', X_test_std)

save('/content/drive/MyDrive/Models/y_train.npy', y_train)
save('/content/drive/MyDrive/Models/y_test.npy', y_test)
     

In [ ]:
#fine Tuning
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Load the dataset which is already processed
# load numpy array from npy file
from numpy import load

X_train_std = load('/content/drive/MyDrive/Models/X_train_std.npy')
X_test_std = load('/content/drive/MyDrive/Models/X_test_std.npy')

y_train = load('/content/drive/MyDrive/Models/y_train.npy')
y_test = load('/content/drive/MyDrive/Models/y_test.npy')
     

In [ ]:
print("X_train_std_shape: {}".format(X_train_std.shape))
print("X_test_std_shape: {}".format(X_test_std.shape))
     

In [ ]:
# Load the saved best model

model_FineTune = keras.models.load_model('/content/drive/MyDrive/Models/01_Xception_TransferLearning_Best_Model.h5')
     

In [ ]:
model_FineTune.summary()


In [ ]:
model_FineTune.layers

In [ ]:
len(model_FineTune.layers)
     

In [ ]:
# Fine-tune the model from index 60 onwards

for layer in model_FineTune.layers[:60]:
  layer.trainable = False

for layer in model_FineTune.layers[60:]:
  layer.trainable = True

In [ ]:
Compile and train the model
Save the best weights while training
#model_FineTune.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])

callbacks_FineTune = [            
             keras.callbacks.ModelCheckpoint("bestFT.h5",
                                             monitor='val_accuracy',
                                             save_weights_only=True,
                                             save_best_only=True)
]

history_FineTune = model_FineTune.fit(x = X_train_std, y = y_train, epochs=50,
                                      validation_split=0.1, batch_size=16, callbacks=callbacks_FineTune)